In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../input/bitcoin-minute-data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd    
import numpy as np     
                       
import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline
from datetime import datetime # for dealing with times

In [ ]:
### Minute by minute of bitcoin price since 2012

df = pd.read_csv('../input/d/tizzoks/bitcoin-minute-data/bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv')
df.head()

In [ ]:
# last 300 minutes of the dataframe

df.tail(300)

In [ ]:
## I have a wide screen monitor so this widens the window.

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
df.columns = df.columns.str.lower()
df['date_time'] = pd.to_datetime(df['timestamp'],unit='s')
df['timestamp'] = df['timestamp'].astype('int')
df.dropna(inplace=True,axis=0)
df.set_index(keys='date_time',inplace=True)

In [ ]:
#  I have a file that stores the results of my runs so I can reference them in the future.
###df_test = pd.read_excel('parameters.xlsx')
###df_test.drop('Unnamed: 0',inplace=True,axis=1)
###df_test.head()

#df_test = pd.DataFrame(columns=['minutes_run','rolling_long','rolling_short','sell_off_percent','buy_in_percent',
#                               'buy_in_ratio','sell_count','liquidation_count','leverage','account','total_money'])


In [ ]:

#### Dataframes that will be used to chart out the price points and markers at the very end of the script
df_all = pd.DataFrame(columns=['price'])
df_liq = pd.DataFrame(columns=['liq_price'])
df_bought = pd.DataFrame(columns=['bought'])
df_sold = pd.DataFrame(columns=['sold_price'])


##### Variable parameters below this line. We can play with these numbers
rolling_long = 30   # Minutes for the long rolling average. Must be higher number than rolling_short
rolling_short = 5   # Minutes for the short rolling average. Must be lower than the high
sell_off_percent = 0.0012   # % gain we want to sell off at
buy_in_percent = 0.0045  # how much below the long avg the short avg needs to be when it crosses.
buy_in_ratio = 0.25      # Percentage of account that you'd like to make purchases at.
account = 1000       # Starting account balance
leverage = 15 # This is the muliplier. Any number greater than 0. 10x means your profits or losses will be 10x greater
###### Parameters above this line.

### Variables used in calculations
buy_in_at = 0
sell_at_value = 0
liquidation_count = 0
liq_point = 0
payout = 0
stock_ratio_bought = 0
stock_ratio_sold = 0
amount_of_stock_bought = 0
from_account = 0
sell_count = 0
minutes_run = 0
holding = False



# Rolling avg values and the line we want to buy in at.
df['long_avg'] = df['open'].rolling(window=rolling_long).mean()
df['short_avg'] = df['open'].rolling(window=rolling_short).mean()
df['buy_line'] = df['long_avg'] - (df['long_avg'] * buy_in_percent)


## In this for loop I chose roughly 1 day worth of data to show some buy/sell cycles as well as a liquidation

for i in range (len(df)-98960,len(df)-97500):
    
    # adding a new row to the dataframe by the minute
    minutes_run += 1
    
    #writes Price to dataframe that will be used at end for charting
    new_row = pd.DataFrame([[ df.iloc[i]['open'] ]], columns=['price'], index=[df.index[i]])
    df_all = pd.concat([df_all, pd.DataFrame(new_row)], ignore_index=False)
        
        
        ### Liquidated criteria. If the Open price drops below the liquidation point then the money is gone
    if df.iloc[i]['open'] <= liq_point:
        holding = False
        liquidation_count += 1
        liq_point = 0
        print('Liquidated with Date and Price Point')
        print(df.index[i])
        print(df.iloc[i]['open'])
        print('\n')
      
        new_row = pd.DataFrame([[ df.iloc[i]['open'] ]], columns=['liq_price'], index=[df.index[i]])
        df_liq = pd.concat([df_liq, pd.DataFrame(new_row)], ignore_index=False)
        
        
        ## Buying trigger. Must not be holding. The buy line must cross the short average from below. The short average must have positive slope. 
    elif holding == False and df.iloc[i-1]['buy_line'] >= df.iloc[i-1]['short_avg'] and df.iloc[i]['buy_line'] <= df.iloc[i]['short_avg'] and df.iloc[i-1]['short_avg'] <= df.iloc[i]['short_avg']:
        print('-----Buy Cycle Beginning-----')
        # The amount of stock we are buying
        amount_of_stock_bought = (account*buy_in_ratio*leverage)/ df.iloc[i]['open']
        # The amount of money we are pulling out to buy
        from_account = account*buy_in_ratio
        # Removing the money from our account
        account = account - (account*buy_in_ratio)
        # Price we are buying at
        buy_in_at = df.iloc[i]['open']
        # Specifying the sell point we plan on 
        sell_at_value = df.iloc[i]['open'] + df.iloc[i]['open']*sell_off_percent
        # Designating that we are now holding
        holding = True
        # Specifying the point at which our holdings will be liquidated
        liq_point = df.iloc[i]['open'] - (df.iloc[i]['open']*(1/leverage))
        
        new_row = pd.DataFrame([[ df.iloc[i]['open'] ]], columns=['bought'], index=[df.index[i]])
        df_bought = pd.concat([df_bought, pd.DataFrame(new_row)], ignore_index=False)
        
        print('pulling money from account worth $' + str(from_account))
        print('amount of stock bought is ' + str(amount_of_stock_bought))
        print('bought in at ' + str(buy_in_at))
        print('We will sell at ' + str(sell_at_value))
        print('Account value at $' + str(account))
        print('bought something on date ' + str(df.index[i]) + ' for ' + str(df.iloc[i]['open']) + ' and will sell at ' + str(sell_at_value) + ' We will Liquidate if it drops to ' + str(liq_point))
        print(' ****** Buy Cycle Ending *****')
        print('\n')
          
            
    #### Selling Cycle        
    elif holding == True and sell_at_value <= df.iloc[i]['open']:
        sell_count += 1
        profit = ((df.iloc[i]['open'] * amount_of_stock_bought) - (buy_in_at * amount_of_stock_bought))
        account  =  account + from_account + profit
        holding = False
        
        new_row = pd.DataFrame([[ df.iloc[i]['open'] ]], columns=['sold_price'], index=[df.index[i]])
        df_sold = pd.concat([df_sold, pd.DataFrame(new_row)], ignore_index=False)
        
        print('$$$$$$$ Selling Cycle beginning $$$$$$$')
        print('Money that went in was ' + str(from_account) + ' and profit is ' + str(profit))
        print('account is $' + str(account))

        print('sold something on date '+ str(df.index[i]) + ' for ' + str(df.iloc[i]['open']) + '. We have gained ' + str(profit) + ' and my account has ' + str(account))
        print('&&&&&&Selling Cycle is ending&&&&&&&&&&')
        print('\n')

print(account)
print(sell_count)
print(liquidation_count)
total_money = from_account + account

### Entering parameters in to separate dataframe for historical reference. Not including that file
##  df_test.loc[len(df_test.index)] = [ minutes_run , rolling_long ,  rolling_short , sell_off_percent , buy_in_percent , buy_in_ratio , sell_count , liquidation_count , leverage , account , total_money]

In [ ]:
## Adding the Liq, bought, sold, and bitcoin prices to a dataframe

df_alls = df_all.join(df_liq, lsuffix='Price',rsuffix='liq')
df_alls = df_alls.join(df_bought, lsuffix='Price',rsuffix='bought')
df_alls = df_alls.join(df_sold, lsuffix='Price',rsuffix='sold')

In [ ]:
### Plotting the price points to plotly
#import plotly.graph_objects as go

import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [ ]:
fig = go.FigureWidget()

fig.add_trace(go.Scatter(x=df_alls.index, y=df_alls['price'],
                          mode='lines',
                          name='Price'))
fig.add_trace(go.Scatter(x=df_alls.index, y=df_alls['liq_price'],
                          mode='markers',
                          name='Liq_Price'))
fig.add_trace(go.Scatter(x=df_alls.index, y=df_alls['bought'],
                          mode='markers',
                          name='bought'))
fig.add_trace(go.Scatter(x=df_alls.index, y=df_alls['sold_price'],
                          mode='markers',
                          name='sold_price'))
fig.show()